In [30]:
# -------------------------------------------------
# File evaluation - Roberto Schlatter
# 15-10-2020
#-------------------------------------------------
import pandas as pd
import os
import matplotlib.pyplot as plt
    
rootdir = '/Users/SCHLARO4/1_PRIVATE/Prog/Private/'

sFilename = 'transactions-from-01012010-to-10112020.csv'

df_transactions = pd.read_csv(sFilename, sep=';', encoding= 'unicode_escape')

lst_header = ['Transaktionen', 'Name']

#----------
# DIVIDENDE
#----------
# Filter table to show only columns "Transaktionen", "Name"
df_dividende = df_transactions[df_transactions['Transaktionen'].isin(['Dividende'])]

df_dividende['Nettobetrag in der Währung des Kontos'] = pd.to_numeric(df_dividende['Nettobetrag in der Währung des Kontos'])

# Final dataframe for dividends
df_export_dividende = df_dividende.groupby(lst_header)['Nettobetrag in der Währung des Kontos'].sum()
print(df_export_dividende)

df_export_dividende.to_excel('nettodividende.xlsx')
#print("Dividende: ", df_export_dividende)

#-------
#KOSTEN
#-------
# KAUF & VERKAUF Kosten
df_kosten_total = df_transactions[df_transactions['Transaktionen'].isin(['Kauf', 'Verkauf'])]

#print(df_kosten_total)
# get 4th entry of row = SYMBOL
#print(df_kosten)

# Final dataframe for costs
df_export_kosten_total = df_kosten_total.groupby(lst_header)['Kosten'].sum()
print(df_export_kosten_total)

df_export_kosten_total.to_excel('kosten.xlsx')
#print(df_export_kosten_total)
#print("total kosten: ", df_export_kosten)



Transaktionen  Name                          
Dividende      AMERICAN EXPRESS ORD                27.39
               AMS                                 38.08
               APPLE ORD                          161.00
               ISHARES 20 PLS YEAR TREASURY B     152.56
               MICROSOFT ORD                       42.12
               NVIDIA ORD                          12.82
               PARTNERS GROUP N                   220.35
               QUALCOMM ORD                       101.66
               ROCHE GS                          1113.12
               SCHWEIZER ELECTRONIC N ORD          49.77
               SPDR S&P 500 ETF                    43.50
               TAIWAN SEMICONDUCTOR MNFTG ADR      42.27
               UBSETF SMIM CHF DIS                316.92
               VANGUARD LONG TERM BOND ETF         30.88
               VANGUARD TOTAL STOCK MARKET ET     245.41
               Vanguard All World ETF Dist        199.91
               Vanguard EM ETF Dist       

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
value = -1
filenames = []
myvals = []
mydate = []
mycycle = []
mytime = []
myfolder = []
myoffset = []
rootdir = '/Users/SCHLARO4/1_PRIVATE/Prog/Private/'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if not filepath.endswith(".csv"):
            continue
        
        filenames.append(file) 
        track = subdir.split('/')
        myfolder.append(track[len(track)-1])
        with open(filepath, 'r') as ifs:
            for line in ifs:
                line = line.strip()
               
                if line.startswith('Date'):
                    tokens = line.split(';')
                    date = tokens[1]
                    mydate.append(date)     
                if line.startswith('Time'):
                    tokens = line.split(';')
                    time = tokens[1]
                    mytime.append(time)
                
                if line.startswith('Cycle number'):
                    tokens = line.split(';')
                    cycle = int(tokens[1])
                    mycycle.append(cycle)
                
                if line.startswith('Tara Y offset'):
                    tokens = line.split(';')
                    offset = tokens[1]
                    myoffset.append(offset)
                                
                if line.startswith('Peak-peak Y'):             
                    # Here we have the right line
                    tokens = line.split(';')
                    value = float(tokens[1].replace(',', '.'))
                    myvals.append(value)
                    break
# combine all lists in one tuple                    
tuples_combined = list(zip(myfolder, filenames, myvals, myoffset, mydate, mytime, mycycle))         

# create dataframe from tuple
df_whole = pd.DataFrame(tuples_combined, columns = ['Track','Filenames','Peak-peak Y [N]', 'Offset Y [N]','Date','Time','Cycle'] )

# provide summary of statistics grouped by "track" column
df_stat = df_whole.groupby('Track')['Peak-peak Y [N]'].describe()

# store statistics summary in Excel
df_stat.to_excel('summary_results.xlsx')

# store whole dataframe in Excel
df_whole.to_excel('results.xlsx')

print(df_stat)

plt = df_whole['Peak-peak Y [N]']
plt.show()

AttributeError: 'Index' object has no attribute 'remove_unused_levels'